In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/who-is-the-real-winner/sample_submission.csv
/kaggle/input/who-is-the-real-winner/train.csv
/kaggle/input/who-is-the-real-winner/test.csv


In [46]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Step 1: Data Preprocessing
data = pd.read_csv('../input/who-is-the-real-winner/train.csv')
data1 = pd.read_csv('../input/who-is-the-real-winner/test.csv')
data.drop(columns=['ID','Candidate','Constituency ∇'], inplace=True)
data1.drop(columns=['ID','Candidate','Constituency ∇'], inplace=True)

def convert_to_lakhs(value):
    value = str(value)
    if value == "0":
        return 0
    elif 'Crore+' in value:
        number, unit = value.split()
        number = float(number.replace(",", ""))
        return int(number * 100)
    elif 'Lac+' in value:
        number, unit = value.split()
        number = float(number.replace(",", ""))
        return int(number)
    elif 'Thou+' in value:
        number, unit = value.split()
        number = float(number.replace(",", ""))
        return int(number / 100)
    elif 'Hund+' in value:
        number, unit = value.split()
        number = float(number.replace(",", ""))
        return int(number / 1000)
    else:
        return int(value)

data["Total Assets"] = data["Total Assets"].apply(convert_to_lakhs)
data["Liabilities"] = data["Liabilities"].apply(convert_to_lakhs)
data1["Total Assets"] = data1["Total Assets"].apply(convert_to_lakhs)
data1["Liabilities"] = data1["Liabilities"].apply(convert_to_lakhs)

# Perform label encoding for Education column

# from sklearn.preprocessing import LabelEncoder

# Encode the target variable
# label_encoder = LabelEncoder()
# data['Education'] = label_encoder.fit_transform(data['Education'])

# data['Education'] = data['Education'].astype('category').cat.codes

data = pd.get_dummies(data, columns=['Party','state'])
data1 = pd.get_dummies(data1, columns=['Party','state'])

data["Diff"] = data["Total Assets"] - data["Liabilities"]
data1["Diff"] = data1["Total Assets"] - data1["Liabilities"]
# data.drop(columns = ['Total Assets','Liabilities'], inplace = True)

In [47]:
X = data.drop(columns=['Education'])
y = data['Education']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# scaler = StandardScaler()
# X = scaler.fit_transform(X)
# data1 = scaler.transform(data1)


rf_classifier = RandomForestClassifier(
    n_estimators=85,
    # criterion='gini',
    max_depth=29,
    min_samples_split=7,
    min_samples_leaf=1,
    # min_weight_fraction_leaf=0.0,
    max_features=2,
    # max_leaf_nodes=None,
    # min_impurity_decrease=0.0,
    # bootstrap=True,
    # oob_score=False,
    # n_jobs=None,
    random_state=42,
    # verbose=1,
    # warm_start=False,
    # class_weight=None,
    # ccp_alpha=0.0,
    # max_samples=None
    )

In [48]:
rf_classifier.fit(X, y)

# Step 3: Model Evaluation
y_pred = rf_classifier.predict(data1)
# f1 = f1_score(y_test, y_pred, average='weighted')
# print("F1 Score:", f1)

In [50]:
df_pred = pd.DataFrame(y_pred, columns=["Education"])
df_pred.to_csv('submission234.csv', index_label='ID')

y_pred

array(['12th Pass', 'Graduate Professional', 'Graduate', ..., 'Graduate',
       'Post Graduate', 'Post Graduate'], dtype=object)